# Lesson 4: Utilizing Redis Streams for Event Logging

# Utilizing Redis Streams for Event Logging

Welcome! In this unit, we will explore how to use Redis streams for event logging. This is a crucial part of our Redis-based backend system project. By the end of this lesson, you’ll know how to log events and retrieve them using Redis streams. So far, you’ve learned to manage user data and handle transactions; now, we’re adding another layer to our system by using streams.

## What You’ll Build

In this unit, we will focus on the following tasks:

1. **Adding entries to a stream**: We’ll log user activities in a Redis stream.
2. **Reading entries from a stream**: You’ll learn to read the logged events from the stream.

Let’s start by refreshing our skills with adding data. This time, we’ll use streams instead of simple keys.

### Code Example

Here’s a snippet to show how you can add an entry to a stream and read it back:

```python
import redis

# Connect to Redis
client = redis.Redis(host='localhost', port=6379, db=0)

# Define stream name
stream_name = 'user_activity_stream'

# Add entries to the stream
client.xadd(stream_name, {'event': 'login', 'username': 'alice'})
client.xadd(stream_name, {'event': 'login', 'username': 'bob'})

# Read entries from the stream
entries = client.xread({stream_name: '0-0'}, count=2)

# Print each entry
for entry in entries:
    print(f"Stream entry: {entry}")
```

In this code:
- **Adding Entries**: We log each user login event to the `user_activity_stream` with the `xadd` method.
- **Reading Entries**: We use `xread` to read entries, where `stream_name` and `count` control the stream reading operation.

Ready to try it yourself? Head over to the practice section and start working on logging events using Redis streams. Happy coding!



## Check to Avoid Duplicate Users

Great job so far! Before adding streams, let's add a small check to see if the user already exists in the database. If the user already exists, we should skip adding the user to the database and move to the next user.

```py
import redis
import json
from datetime import timedelta

client = redis.Redis(host='localhost', port=6379, db=0)

def add_user(user_id, data, pipeline=None):
    if pipeline:
        pipeline.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))
    else:
        client.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))

def get_user(user_id):
    data = client.get(f'user:{user_id}')
    return json.loads(data) if data else None

def add_score(user_id, score, pipeline=None):
    if pipeline:
        pipeline.zadd('leaderboard', {user_id: score})
    else:
        client.zadd('leaderboard', {user_id: score})

def get_leaderboard(top_n=10):
    leaderboard = client.zrevrange('leaderboard', 0, top_n - 1, withscores=True)
    return [(user_id.decode('utf-8'), score) for user_id, score in leaderboard]

def get_user_rank_and_score(user_id):
    rank = client.zrevrank('leaderboard', user_id)
    score = client.zscore('leaderboard', user_id)
    return rank, score

users = [
    {'user_id': 'alice', 'data': {'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}, 'score': 50},
    {'user_id': 'bob', 'data': {'name': 'Bob', 'age': 25, 'email': 'bob@example.com'}, 'score': 80},
    {'user_id': 'charlie', 'data': {'name': 'Charlie', 'age': 35, 'email': 'charlie@example.com'}, 'score': 70}
]

with client.pipeline() as pipeline:
    for user in users:
        # TODO: Check if the user exists, if so, skip adding the user

        add_user(user['user_id'], user['data'], pipeline)
        add_score(user['user_id'], user['score'], pipeline)
    pipeline.execute()

users.append({'user_id': 'david', 'data': {'name': 'David', 'age': 40, 'email': 'david@example.com'}, 'score': 90})

add_user(users[-1]['user_id'], users[-1]['data'])
add_score(users[-1]['user_id'], users[-1]['score'])

print("Leaderboard:", get_leaderboard())

for user in users:
    rank, score = get_user_rank_and_score(user['user_id'])
    print(f"User {user['user_id']} rank: {rank}, score: {score}")
    
```

Here’s how you can modify the code to include a check for existing users before adding them. The `add_user` function will first call `get_user` to check if the user already exists. If the user exists, the function will skip adding that user and continue with the next.

```python
import redis
import json
from datetime import timedelta

client = redis.Redis(host='localhost', port=6379, db=0)

def add_user(user_id, data, pipeline=None):
    # Check if the user already exists
    if get_user(user_id):
        print(f"User {user_id} already exists, skipping...")
        return
    if pipeline:
        pipeline.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))
    else:
        client.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))

def get_user(user_id):
    data = client.get(f'user:{user_id}')
    return json.loads(data) if data else None

def add_score(user_id, score, pipeline=None):
    if pipeline:
        pipeline.zadd('leaderboard', {user_id: score})
    else:
        client.zadd('leaderboard', {user_id: score})

def get_leaderboard(top_n=10):
    leaderboard = client.zrevrange('leaderboard', 0, top_n - 1, withscores=True)
    return [(user_id.decode('utf-8'), score) for user_id, score in leaderboard]

def get_user_rank_and_score(user_id):
    rank = client.zrevrank('leaderboard', user_id)
    score = client.zscore('leaderboard', user_id)
    return rank, score

# User data to be added
users = [
    {'user_id': 'alice', 'data': {'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}, 'score': 50},
    {'user_id': 'bob', 'data': {'name': 'Bob', 'age': 25, 'email': 'bob@example.com'}, 'score': 80},
    {'user_id': 'charlie', 'data': {'name': 'Charlie', 'age': 35, 'email': 'charlie@example.com'}, 'score': 70}
]

# Add users and scores in a pipeline
with client.pipeline() as pipeline:
    for user in users:
        add_user(user['user_id'], user['data'], pipeline)
        add_score(user['user_id'], user['score'], pipeline)
    pipeline.execute()

# Add a new user 'david' individually
users.append({'user_id': 'david', 'data': {'name': 'David', 'age': 40, 'email': 'david@example.com'}, 'score': 90})
add_user(users[-1]['user_id'], users[-1]['data'])
add_score(users[-1]['user_id'], users[-1]['score'])

# Display leaderboard
print("Leaderboard:", get_leaderboard())

# Print each user’s rank and score
for user in users:
    rank, score = get_user_rank_and_score(user['user_id'])
    print(f"User {user['user_id']} rank: {rank}, score: {score}")
```

### Explanation of Changes
1. **Check if User Exists**: In the `add_user` function, `get_user(user_id)` checks if the user data is already stored. If the user exists, a message is printed, and the function returns without adding the user to Redis.
2. **Pipeline Execution**: `pipeline` adds or updates users and scores only for users who don’t exist yet, optimizing the Redis transactions.

Now, the code will add new users and skip those already stored.

## Logging User Activities with Redis

Great job so far!

Next, let's create a user activity stream and log user activities to it. We'll use Redis streams to store user activities.

Follow the TODO instructions in the starter code to complete the implementation.

```py
import redis
import json
from datetime import timedelta

client = redis.Redis(host='localhost', port=6379, db=0)

def add_user(user_id, data, pipeline=None):
    if pipeline:
        pipeline.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))
    else:
        client.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))

def get_user(user_id):
    data = client.get(f'user:{user_id}')
    return json.loads(data) if data else None

def add_score(user_id, score, pipeline=None):
    if pipeline:
        pipeline.zadd('leaderboard', {user_id: score})
    else:
        client.zadd('leaderboard', {user_id: score})

def get_leaderboard(top_n=10):
    leaderboard = client.zrevrange('leaderboard', 0, top_n - 1, withscores=True)
    return [(user_id.decode('utf-8'), score) for user_id, score in leaderboard]

def get_user_rank_and_score(user_id):
    rank = client.zrevrank('leaderboard', user_id)
    score = client.zscore('leaderboard', user_id)
    return rank, score

# TODO: Implement add_to_stream function that takes stream_name and entry as arguments
    # TODO: Add entry to stream named `stream_name`

users = [
    {'user_id': 'alice', 'data': {'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}, 'score': 50},
    {'user_id': 'bob', 'data': {'name': 'Bob', 'age': 25, 'email': 'bob@example.com'}, 'score': 80},
    {'user_id': 'charlie', 'data': {'name': 'Charlie', 'age': 35, 'email': 'charlie@example.com'}, 'score': 70}
]

# TODO: Define stream name for user activities 'user_activity_stream'

with client.pipeline() as pipeline:
    for user in users:
        user_obj = get_user(user['user_id'])
        if user_obj:
            continue
        add_user(user['user_id'], user['data'], pipeline)
        add_score(user['user_id'], user['score'], pipeline)
        # TODO: Call the add_to_stream function to add user activity to the stream with the following fields:
            # event: 'user_added'
            # user_id: user['user_id']
            # data: json.dumps(user['data'])
            # score: user['score']
    pipeline.execute()

users.append({'user_id': 'david', 'data': {'name': 'David', 'age': 40, 'email': 'david@example.com'}, 'score': 90})

add_user(users[-1]['user_id'], users[-1]['data'])
add_score(users[-1]['user_id'], users[-1]['score'])

print("Leaderboard:", get_leaderboard())

for user in users:
    rank, score = get_user_rank_and_score(user['user_id'])
    print(f"User {user['user_id']} rank: {rank}, score: {score}")



```

To implement a Redis stream for logging user activities, we'll add a `user_activity_stream` and define a new function called `add_to_stream`. This function will add entries to the stream, including fields like `event`, `user_id`, `data`, and `score` for tracking user activities. Here’s the updated code with comments for each TODO item.

```python
import redis
import json
from datetime import timedelta

client = redis.Redis(host='localhost', port=6379, db=0)

def add_user(user_id, data, pipeline=None):
    if pipeline:
        pipeline.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))
    else:
        client.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))

def get_user(user_id):
    data = client.get(f'user:{user_id}')
    return json.loads(data) if data else None

def add_score(user_id, score, pipeline=None):
    if pipeline:
        pipeline.zadd('leaderboard', {user_id: score})
    else:
        client.zadd('leaderboard', {user_id: score})

def get_leaderboard(top_n=10):
    leaderboard = client.zrevrange('leaderboard', 0, top_n - 1, withscores=True)
    return [(user_id.decode('utf-8'), score) for user_id, score in leaderboard]

def get_user_rank_and_score(user_id):
    rank = client.zrevrank('leaderboard', user_id)
    score = client.zscore('leaderboard', user_id)
    return rank, score

# Define stream name for user activities
user_activity_stream = 'user_activity_stream'

# Implement add_to_stream function that takes stream_name and entry as arguments
def add_to_stream(stream_name, entry):
    """
    Adds an entry to a Redis stream.
    """
    client.xadd(stream_name, entry)

users = [
    {'user_id': 'alice', 'data': {'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}, 'score': 50},
    {'user_id': 'bob', 'data': {'name': 'Bob', 'age': 25, 'email': 'bob@example.com'}, 'score': 80},
    {'user_id': 'charlie', 'data': {'name': 'Charlie', 'age': 35, 'email': 'charlie@example.com'}, 'score': 70}
]

with client.pipeline() as pipeline:
    for user in users:
        user_obj = get_user(user['user_id'])
        if user_obj:
            continue  # Skip existing users
        add_user(user['user_id'], user['data'], pipeline)
        add_score(user['user_id'], user['score'], pipeline)
        
        # Log user activity to the stream
        add_to_stream(user_activity_stream, {
            'event': 'user_added',
            'user_id': user['user_id'],
            'data': json.dumps(user['data']),
            'score': user['score']
        })
    pipeline.execute()

# Adding a new user 'david' individually without pipeline
users.append({'user_id': 'david', 'data': {'name': 'David', 'age': 40, 'email': 'david@example.com'}, 'score': 90})

add_user(users[-1]['user_id'], users[-1]['data'])
add_score(users[-1]['user_id'], users[-1]['score'])

# Log David's addition to the activity stream
add_to_stream(user_activity_stream, {
    'event': 'user_added',
    'user_id': users[-1]['user_id'],
    'data': json.dumps(users[-1]['data']),
    'score': users[-1]['score']
})

print("Leaderboard:", get_leaderboard())

for user in users:
    rank, score = get_user_rank_and_score(user['user_id'])
    print(f"User {user['user_id']} rank: {rank}, score: {score}")
```

### Explanation of Additions:
1. **Stream Name**: `user_activity_stream` is defined for logging user activities.
2. **`add_to_stream` Function**: This function accepts `stream_name` and `entry` parameters and logs the user’s activities to the specified stream.
3. **Logging User Activities**: Inside the main loop and after adding David individually, `add_to_stream` logs relevant details to `user_activity_stream` whenever a new user is added.

With this setup, user activities like additions are now logged in the `user_activity_stream`, making it easier to track user events.

## Logging Activities in Redis Streams

Great job so far!

Now let's add another feature for reading entries from the stream. You need to implement the read_from_stream function that reads entries from the stream starting from the beginning. Follow the instructions in the code to complete the task.

```py
import redis
import json
from datetime import timedelta

client = redis.Redis(host='localhost', port=6379, db=0)

def add_user(user_id, data, pipeline=None):
    if pipeline:
        pipeline.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))
    else:
        client.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))

def get_user(user_id):
    data = client.get(f'user:{user_id}')
    return json.loads(data) if data else None

def add_score(user_id, score, pipeline=None):
    if pipeline:
        pipeline.zadd('leaderboard', {user_id: score})
    else:
        client.zadd('leaderboard', {user_id: score})

def get_leaderboard(top_n=10):
    leaderboard = client.zrevrange('leaderboard', 0, top_n - 1, withscores=True)
    return [(user_id.decode('utf-8'), score) for user_id, score in leaderboard]

def get_user_rank_and_score(user_id):
    rank = client.zrevrank('leaderboard', user_id)
    score = client.zscore('leaderboard', user_id)
    return rank, score

def add_to_stream(stream_name, entry):
    client.xadd(stream_name, entry)

# TODO: Implement the read_from_stream function that takes stream_name as input
    # TODO: Read entries from the stream starting from the beginning

users = [
    {'user_id': 'alice', 'data': {'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}, 'score': 50},
    {'user_id': 'bob', 'data': {'name': 'Bob', 'age': 25, 'email': 'bob@example.com'}, 'score': 80},
    {'user_id': 'charlie', 'data': {'name': 'Charlie', 'age': 35, 'email': 'charlie@example.com'}, 'score': 70}
]

# Stream name for user activities
stream_name = 'user_activity_stream'

with client.pipeline() as pipeline:
    for user in users:
        user_obj = get_user(user['user_id'])
        if user_obj:
            continue
        add_user(user['user_id'], user['data'], pipeline)
        add_score(user['user_id'], user['score'], pipeline)
        add_to_stream(stream_name, {
            'event': 'user_added',
            'user_id': user['user_id'],
            'data': json.dumps(user['data']),
            'score': user['score']
        })
    pipeline.execute()

users.append({'user_id': 'david', 'data': {'name': 'David', 'age': 40, 'email': 'david@example.com'}, 'score': 90})

add_user(users[-1]['user_id'], users[-1]['data'])
add_score(users[-1]['user_id'], users[-1]['score'])

print("Leaderboard:", get_leaderboard())

for user in users:
    rank, score = get_user_rank_and_score(user['user_id'])
    print(f"User {user['user_id']} rank: {rank}, score: {score}")

# TODO: Read entries from the stream using the read_from_stream function

# TODO: Iterate over the stream entries and print each entry



```

Here’s the completed implementation of the `read_from_stream` function, which reads entries from the beginning of the stream. This function will help you retrieve all the entries logged in the `user_activity_stream`, and we’ll loop through each entry to print it.

```python
import redis
import json
from datetime import timedelta

client = redis.Redis(host='localhost', port=6379, db=0)

def add_user(user_id, data, pipeline=None):
    if pipeline:
        pipeline.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))
    else:
        client.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))

def get_user(user_id):
    data = client.get(f'user:{user_id}')
    return json.loads(data) if data else None

def add_score(user_id, score, pipeline=None):
    if pipeline:
        pipeline.zadd('leaderboard', {user_id: score})
    else:
        client.zadd('leaderboard', {user_id: score})

def get_leaderboard(top_n=10):
    leaderboard = client.zrevrange('leaderboard', 0, top_n - 1, withscores=True)
    return [(user_id.decode('utf-8'), score) for user_id, score in leaderboard]

def get_user_rank_and_score(user_id):
    rank = client.zrevrank('leaderboard', user_id)
    score = client.zscore('leaderboard', user_id)
    return rank, score

def add_to_stream(stream_name, entry):
    client.xadd(stream_name, entry)

# Implement the read_from_stream function
def read_from_stream(stream_name):
    """
    Reads entries from the Redis stream starting from the beginning.
    """
    entries = client.xread({stream_name: '0-0'})
    return entries

users = [
    {'user_id': 'alice', 'data': {'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}, 'score': 50},
    {'user_id': 'bob', 'data': {'name': 'Bob', 'age': 25, 'email': 'bob@example.com'}, 'score': 80},
    {'user_id': 'charlie', 'data': {'name': 'Charlie', 'age': 35, 'email': 'charlie@example.com'}, 'score': 70}
]

# Stream name for user activities
stream_name = 'user_activity_stream'

with client.pipeline() as pipeline:
    for user in users:
        user_obj = get_user(user['user_id'])
        if user_obj:
            continue
        add_user(user['user_id'], user['data'], pipeline)
        add_score(user['user_id'], user['score'], pipeline)
        add_to_stream(stream_name, {
            'event': 'user_added',
            'user_id': user['user_id'],
            'data': json.dumps(user['data']),
            'score': user['score']
        })
    pipeline.execute()

users.append({'user_id': 'david', 'data': {'name': 'David', 'age': 40, 'email': 'david@example.com'}, 'score': 90})

add_user(users[-1]['user_id'], users[-1]['data'])
add_score(users[-1]['user_id'], users[-1]['score'])

print("Leaderboard:", get_leaderboard())

for user in users:
    rank, score = get_user_rank_and_score(user['user_id'])
    print(f"User {user['user_id']} rank: {rank}, score: {score}")

# Read entries from the stream
entries = read_from_stream(stream_name)

# Iterate over the stream entries and print each entry
for entry_id, entry_data in entries[0][1]:
    print(f"Entry ID: {entry_id}")
    for key, value in entry_data.items():
        print(f"{key.decode('utf-8')}: {value.decode('utf-8')}")
    print()
```

### Explanation:

1. **`read_from_stream` Function**: This function reads all entries from the specified stream starting from the beginning (`'0-0'`).
2. **Iterate and Print Entries**: After calling `read_from_stream`, we iterate over the entries and print each entry’s ID and data in a readable format.

### Output:

Running this code will display each entry in `user_activity_stream` along with its details, providing a clear history of user activities like user additions. This makes it easier to retrieve and inspect user actions in the Redis stream!